# TBCL parser

In [1]:
import os, re, glob, requests, io, urllib, json
import pandas as pd
import opencc

pd.options.display.max_rows = 2000

opencc_tw2s = opencc.OpenCC('tw2s')

Download files from TBCL home page: https://coct.naer.edu.tw/download/tech_report/

In [2]:
![[ ! -d downloads && -d ../downloads/tbcl ]] && ln -s ../downloads/tbcl downloads
!mkdir -p downloads

if not os.path.exists('downloads/.done'):
    home_url = 'https://coct.naer.edu.tw/download/tech_report/'
    resp = requests.get(home_url).content.decode('utf-8')
    for url in sorted(re.findall('<a href="([^"]+[.](?:xlsx|docx))"', resp)):
        url = os.path.join(home_url, url)
        !cd downloads && wget -nc "{url}"
    !chmod a-w downloads/*.xlsx; touch downloads/.done

Symlinks for convenience and checksums:

In [3]:
%%bash -e
cd downloads
chmod a-w *.xlsx *.ods *.pdf
ln -sf '臺灣華語文能力基準詞語表_111-11-14.xlsx' tbcl-words.xlsx
ln -sf '臺灣華語文能力基準詞語表_111-11-14.ods' tbcl-words.ods
ln -sf '臺灣華語文能力基準漢字表_111-09-20.docx' tbcl-chars.docx
ln -sf '臺灣華語文能力基準漢字表_111-09-20.xlsx' tbcl-chars.xlsx
ln -sf '臺灣華語文能力基準類詞綴表_111-09-20.docx' tbcl-affix.docx
ln -sf '臺灣華語文能力基準類詞綴表_111-09-20.xlsx' tbcl-affix.xlsx
ln -sf '臺灣華語文能力基準語法點表_112-01-04.xlsx' tbcl-grammar.xlsx
ln -sf '臺灣華語文能力基準語法點表_112-01-04.docx' tbcl-grammar.docx
ln -sf '臺灣華語文能力基準基礎詞彙表_111-09-20.xlsx' tbcl-glossary.xlsx
sha256sum *.xlsx

8bf588d24cb2f1ca37ce094110457619af5dc72e561fd7ed826bfa2489a58713  8000zhuyin_20160215.xlsx
6b5e4845ac2220de985ff775da5c79a5b04e0055c4a4c7aedecb010c5c826706  8000zhuyin_20160316.xlsx
7a0a7f719ecbdb77f01c3ba29059d74141924ef2732299e368ef922da0120c10  8000zhuyin_20161230.xlsx
8da050a48af2ee39b7a693b689a5c318b0991780d3cadedef391b83bc60cc8b8  8000zhuyin_20170324.xlsx
1c360c3f0addccb967b2d4d922be1afcb25a0f0791a8fc8858f4c01c6d1ab055  8000zhuyin_20180419.xlsx
61f659c3ebdd64362879cd0dc5d85c8b3a28586439fc6941f30ab958bcd76f74  8000zhuyin_202204.xlsx
e979ac6d953fb493502e54536b4b6ff534d06e3938700052aa15806d514efc92  8000zhuyin_202307.xlsx
5e92ac49c5bb203e16fea29c53a2b2cb790033fb332a699c7689adde21528b8f  tbcl-affix.xlsx
6329e2516c5dbe416b85f6a94d200ebe95493f24f233a63dc10d85aa257a088f  tbcl-chars.xlsx
b6ce3747a06c8482ce5f4059689463de01a45d2b78707feb85917404ccffae62  tbcl-glossary.xlsx
c587989cf89992d55d97a2f932289ef071648ca80339ccfaff7bb823914e5bcf  tbcl-grammar.xlsx
cb16dcd262eb3e499273f972c9a3a404c4

## Parse wordlist

In [4]:
glossary_df = pd.read_excel('downloads/tbcl-glossary.xlsx').rename(columns={
    '序號': 'ID',
    '詞語': 'Traditional',
    '注音': 'Zhuyin',
    '漢拼': 'Pinyin',
    '詞類/性質': 'POS',
    '詞彙英譯': 'Meaning',
    '語義/義項': 'Meaning2',
    '用法-常用搭配詞': 'Compounds',
    '例句': 'Examples',
    '級別': 'Level',
})
assert list(glossary_df.ID - 1) == list(glossary_df.index)
glossary_df['Level'] = glossary_df.Level.str.extract('^第([1-7][*]?)級$')[0]
assert sum(glossary_df.Level.isnull()) == 0
glossary_df.to_csv('downloads/tbcl-glossary.csv', index=False)
print('downloads/tbcl-glossary.csv: %d rows' % len(glossary_df))

downloads/tbcl-glossary.csv: 1518 rows


In [5]:
opencc_tw2s = opencc.OpenCC('tw2s')

# Character levels from Table of General Standard Chinese Characters for verification.
tgh_level = pd.read_csv('../chars/tgh.csv').set_index('char').level.to_dict()

# Convert to simplified characters + verify
def to_simplified(trad):
    simp = opencc_tw2s.convert(trad)
    for x, y in ('擡抬', '砲炮', '牠它', '妳你', '姪侄', '瞇眯', '舖铺', '搥捶', '暱昵', '瑯琅'):
        simp = simp.replace(x, y)
    if '/' in simp and len(set(simp.split('/'))) == 1:
        simp = simp.split('/')[0]
    for c in simp:
        assert c in tgh_level or c in '/(),吋拚徬祂', (trad, simp, c)
    return simp

In [6]:
VARIANTS_EXC = {
    ('姊姊/姐姐/姊/姐', 'jiějie/jiě'): [['姊姊','jiějie'], ['姐姐','jiějie'], ['姊','jiě'], ['姐','jiě']],
    ('那/那裡/那裏/那兒', 'nà/nàlǐ/nàr'): [['那', 'nà'], ['那裡', 'nàlǐ'], ['那裏', 'nàlǐ'], ['那兒', 'nàr']],
    ('這/這裡/這裏/這兒', 'zhè/zhèlǐ/zhèr'): [['這', 'zhè'], ['這裡', 'zhèlǐ'], ['這裏', 'zhèlǐ'], ['這兒', 'zhèr']],
    ('手錶/手表/錶/表', 'shǒubiǎo/biǎo'): [['手錶', 'shǒubiǎo'], ['手表', 'shǒubiǎo'], ['錶', 'biǎo'], ['表', 'biǎo']],
    ('新台幣/新臺幣/台幣/臺幣', 'xīntáibì/táibì'): [['新台幣', 'xīntáibì'], ['新臺幣', 'xīntáibì'], ['台幣', 'táibì'], ['臺幣', 'táibì']],
    ('新台幣/新臺幣/台幣/臺幣', 'Xīntáibì/Táibì'): [['新台幣', 'Xīntáibì'], ['新臺幣', 'Xīntáibì'], ['台幣', 'Táibì'], ['臺幣', 'Táibì']],
    ('慾望/欲望/慾', 'yùwàng/yù'): [['慾望', 'yùwàng'], ['欲望', 'yùwàng'], ['慾', 'yù']],
    ('侄子/姪子/侄兒/姪兒', 'zhízi/zhír'): [['侄子', 'zhízi'], ['姪子', 'zhízi'], ['侄兒', 'zhír'], ['姪兒', 'zhír']],
    ('嘴脣/嘴唇/脣/唇', 'zuǐchún/chún'): [['嘴脣', 'zuǐchún'], ['嘴唇', 'zuǐchún'], ['脣', 'chún'], ['唇', 'chún']],
    ('沒(有)用', 'méi(yǒu)yòng'): [['沒用', 'méiyòng'], ['沒有用', 'méiyǒuyòng']],
    ('一邊(兒)', 'yìbiān(r)'): [['一邊', 'yìbiān'], ['一邊兒', 'yìbiānr']],
    ('一邊(兒)', 'yībiān(r)'): [['一邊', 'yībiān'], ['一邊兒', 'yībiānr']],
    ('皺眉頭/皺眉', 'zhòuméi/zhòuméitóu'): [['皺眉頭', 'zhòuméitóu'], ['皺眉', 'zhòuméi']],
}

def get_variants(vocab, pinyin):
    vocab = vocab.strip()
    pinyin = re.sub(' */ *', '/', pinyin.strip())

    ps = re.sub('[^()/]', '', pinyin)
    vs = re.sub('[^()/]', '', vocab)
    if ps == '' and vs == '':
        return []

    if (vocab, pinyin) in VARIANTS_EXC:
        return VARIANTS_EXC[(vocab, pinyin)]

    if vs == '' and ps:
        assert set(ps) == {'/'}
        return [[vocab, p.strip()] for p in pinyin.split('/')]

    if vs and ps == '':
        assert set(vs) == {'/'}
        assert len(set(map(len, vocab.split('/')))) == 1, vocab  # all terms same length
        return [[v, pinyin] for v in vocab.split('/')]

    assert vs == ps and set(ps) == {'/'}, (vocab, pinyin)
    return [[v.strip(), p.strip()] for (v, p) in zip(vocab.split('/'), pinyin.split('/'))]

def get_variants_str(vocab, pinyin, moe):
    variants = get_variants(vocab, pinyin)
    if not variants:
        return ''
    arr = []
    for (trad, py) in variants:
        m = [x for x in json.loads(moe.replace("'", '"')) if x[0] == trad]
        assert len(m) <= 1
        if m:
            assert m[0][0] == trad
            m = ' '.join(m[0][1])
        else:
            m = ''
        arr.append({
            'Traditional': trad,
            'Simplified': to_simplified(trad),
            'Pinyin': normalize_pinyin(py, trad),
            #'PinyinYB': py,
            'MOE': m,
        })
    return json.dumps(arr, ensure_ascii=False)

def fix_traditional(s):
    # number suffixes for different pronunciations, +3 weird duplicate entries 空檔 道 來往
    s = re.sub('[0-9]', '', s)
    s = re.sub('／', '/', s)
    assert re.match('^[\u4E00-\u9FFF/()]+$', s), (row, s)
    return s


def normalize_pinyin(py, hz):
    for x, y in ['ɑa', (' */ *', '/'), (r'\s+', ' '), (' */$', ''), ('^/ *', '')]:
        py = re.sub(x, y, py).strip()

    # Pinyin spaces are not meaningul in TBCL lists, mostly just syllable spaces there.
    # Remove to make more mergeable with TOCFL. Also no upper letters.
    if ' ' in py:
        assert py == py.lower() and "'" not in py, py
        merged = ''
        for part in py.split():
            if merged and merged[-1] not in '/()' and part[0] in 'aeoāáǎàēéěèōóǒò':
                merged += "'"
            merged += part
        py = merged
        assert re.match("^[a-zāáǎàēéěèīíǐìōóǒòūúǔùüǘǚǜ/()']+$", py), (py, repr(py))

    if (py, hz) == ('búzhìyú', '不至於/不致於'): return 'bùzhìyú'
    if (py, hz) == ('yíyìgūxíng', '一意孤行'): return 'yīyìgūxíng'
    if (py, hz) == ('yìxīnyīyì', '一心一意'): return 'yīxīnyīyì'
    if (py, hz) == ('yìxīnyíyì', '一心一意'): return 'yīxīnyīyì'

    if '不' in hz and 'bú' in py:
        assert hz.count('不') == py.count('bú') + py.count('bù'), (py, hz)
        py = py.replace('bú', 'bù')

    if '一' in hz and re.search('(yí|yì)', py):
        assert hz.count('一') == py.count('yí') + py.count('yì'), (py, hz)
        py = py.replace('yí', 'yī')
        py = py.replace('yì', 'yī')

    return py

# Correct some errors in pinyin
pinyin_corr_df = pd.read_csv('errata.csv', comment='#', dtype='str')

def fix_pinyin(py, trad=''):
    for row in pinyin_corr_df.itertuples():
        if row.Pinyin == py and row.Traditional == trad:
            py = row.Corrected
    assert re.match('^[a-zāáǎàēéěèīíǐìōóǒòūúǔùüǘǚǜ/(), \']+$', py.lower()), (py, repr(py))
    return py


df = pd.read_excel('downloads/tbcl-words.xlsx').rename(columns={
    '序號': 'ID',
    '詞語': 'Traditional',
    '等別': 'Grade',
    '級別': 'Level',
    '情境': 'Context',
    '書面字頻(每百萬字)': 'WFreq',
    '口語字頻(每百萬字)': 'SFreq',
    '簡編本系統號': 'MOE', # MOE dict IDs, https://dict.concised.moe.edu.tw/dictView.jsp?ID=.
    '參考注音': 'Zhuyin',
    '參考漢語拼音': 'PinyinYB'  # pinyin with tone change indication for 一 and 不
})

assert list(df.ID - 1) == list(df.index)
df = df.drop(columns=['Grade', 'Zhuyin', 'Context'])

df['Level'] = df.Level.str.extract('^第([1-7][*]?)級$')[0]
assert sum(df.Level.isnull()) == 0

df['glossary_key'] = (df.Level.str.slice(0, 1) + df.Traditional)
glossary_df = glossary_df.fillna('')
glossary_df['glossary_key'] = (glossary_df.Level.str.slice(0, 1) + glossary_df.Traditional)
glossary_mp = glossary_df.assign(idx=glossary_df.index).groupby('glossary_key').idx.apply(list)

df['Traditional'] = df.Traditional.map(fix_traditional)
df.insert(2, 'Simplified', df.Traditional.map(to_simplified))
df['Pinyin'] = [normalize_pinyin(row.PinyinYB, row.Traditional) for row in df.itertuples()]
df['Pinyin'] = [fix_pinyin(row.Pinyin, row.Traditional) for row in df.itertuples()]
del df['PinyinYB']
df['Variants'] = [get_variants_str(row.Traditional, row.Pinyin, row.MOE) for row in df.itertuples()]

for row in df.itertuples():
    variants = [v['Traditional'] for v in json.loads(row.Variants)] if row.Variants else [row.Traditional]
    moe = json.loads(row.MOE.replace("'", '"'))
    if row.Variants or row.MOE == '[]':
        for v, ids in moe:
            assert v in variants
        df.loc[row.Index, 'MOE'] = ''
    else:
        assert len(moe) == 1 and moe[0][0] == row.Traditional
        df.loc[row.Index, 'MOE'] = ' '.join(moe[0][1])

# Join with vocab_df
for col in ['POS', 'Meaning', 'Compounds', 'Examples']:
    df[col] = ''
    for row in df.itertuples():
        text = [glossary_df.loc[i, col] for i in glossary_mp.get(row.glossary_key, [])]
        text = [s.strip() for s in text if s.strip()]
        assert ' / ' not in ''.join(text), text
        if not text: continue
        dedup = []
        for s in text:
            if s not in dedup: dedup.append(s)
        text = ' / '.join(dedup)
        if col == 'POS':
            text = text.replace(' ', '').strip()
            text = text.replace('V/Ｖ', 'V')
            text = text.replace('Phrase', 'Ph')
            text = text.replace('詞組', 'Ph')
            text = text.replace('Conj.', 'Conj')
        elif col == 'Compounds':
            text = text.replace(';', '')
            text = text.replace(' / ', '，').split('，')
            dedup = []
            for s in text:
                if s not in dedup: dedup.append(s)
            text = '，'.join(text)
            if text: text += '。'
        for x, y in [(' *[(] +', ' ('), (' +[)]', ')'), (' +/ +', ' / ')]:
            text = re.sub(x, y, text).strip()
        assert '\n' not in text
        df.loc[row.Index, col] = text.strip()

print('Unjoined glossary vocab: %s' % ' '.join(set(glossary_df.glossary_key) - set(df.glossary_key)))
df = df.drop(columns=['glossary_key'])

for col in ['WFreq', 'SFreq', 'MOE', 'Variants']:
    assert col in df
    oldcol = df[col]
    df = df.drop(columns=[col])
    df[col] = oldcol

df.to_csv('tbcl.csv', index=False)
print('tbcl.csv: %d terms' % len(df))

Unjoined glossary vocab: 3月台/月臺 1英國 1臺灣/台灣 1還 3韓國 1美國 1中國 1車/車子 1日本 1法國 3亞洲 3歐洲 2瓶子/瓶瓶 1應該/應 3非洲 3美洲 2罐 3義大利 2德國
tbcl.csv: 14425 terms


In [7]:
df.head()

,ID,Traditional,Simplified,Level,Pinyin,POS,Meaning,Compounds,Examples,WFreq,SFreq,MOE,Variants
0,1,愛,爱,1,ài,Vs/N,to love / love,,我愛你。他很愛吃水果。 / 爸爸媽媽對孩子的愛很多。,535,681,39542,
1,2,吧,吧,1,ba,Ptc,"final particle (for confirming, request)",走吧。,你是從英國來的吧？我們一起去運動吧！,706,748,32 103,
2,3,八,八,1,bā,N,eight,八個，八張，八年。,他買了八個包子。,214,163,1,
3,4,爸爸/爸,爸爸/爸,1,bàba/bà,N,father,,我爸爸喜歡看電影。爸爸，生日快樂。,226,806,,"[{""Traditional"": ""爸爸"", ""Simplified"": ""爸爸"", ""Pi..."
4,5,百,百,1,bǎi,N,hundred,一百個人，三百六十五天，一百五十分鐘，一百塊。,一年有三百六十五天。,108,77,157 334,


In [8]:
# Generate version with variants expanded.

expanded_rows = []
for row in df.fillna('').to_dict(orient='records'):
    variants = json.loads(row['Variants']) if row['Variants'] else [{}]
    for variant in variants:
        var = dict(row)
        var.update(variant)
        expanded_rows.append(var)

expanded_df = pd.DataFrame(expanded_rows).drop(columns=['Variants'])
expanded_df.to_csv('expanded/tbcl.csv', index=False)
print('expanded/tbcl.csv: %d terms' % len(expanded_df))

expanded/tbcl.csv: 14868 terms


In [9]:
EAC1_TAG = '\uEAC1\uEC00\uEC00\uECCC\uEC99'  # tag color, #00cc99 green
EAC1_EX  = '\uEAC1\uEC00\uEC05\uECAA\uECFF'  # examples, #05aaff blue
EAC1_HL  = '\uEAC1\uEC00\uEC00\uECCC\uECCC'  # term highlight in examples, teal

with open('pleco/tbcl-pleco.txt', 'w') as fout:
    last_header = ''
    for row in pd.read_csv('tbcl.csv', dtype='str').fillna('').to_dict(orient='records'):
        header = f"//TBCL/Level {row['Level']}"
        if header != last_header:
            last_header = header
            fout.write(header + '\n')

        variants = json.loads(row['Variants']) if row['Variants'] else [{}]
        for variant in variants:
            var = dict(row)
            var.update(variant)
            defn = ' '.join([
                f"{row['Traditional']} [{row['Pinyin']}]\uEAB1" if row['Variants'] else '',
                f"({row['POS']})" if row.get('POS') else '',
                f"{row['Meaning']}" if row.get('Meaning') else '',
                f"{EAC1_TAG}[TBCL{row['Level']}]\uEAC2",
            ])
            defn = re.sub(r'\s+', ' ', defn).replace('\uEAB1 ', '\uEAB1').strip()
            # Compounds and examples in light blue on separate lines
            for ex in [var['Compounds'], var['Examples']]:
                if not ex: continue
                defn += (
                    f'\uEAB1{EAC1_EX}' +
                    ex.replace(' / ', '\uEAB1').replace(
                        var['Traditional'],
                        f"\uEAC2{EAC1_HL}{var['Traditional']}\uEAC2{EAC1_EX}"
                    ) +
                    '\uEAC2'
                )
            key = f"{var['Simplified']}[{var['Traditional']}]\t{var['Pinyin']}"
            fout.write(f'{key}\t{defn}\n')

!ls -l pleco/tbcl-pleco.txt

-rw-r--r-- 1 jovyan users 1121374 Apr 15 00:19 pleco/tbcl-pleco.txt


## Convert other files

In [10]:
df = pd.read_excel('downloads/tbcl-chars.xlsx').rename(columns={
    '序號': 'ID',
    '漢字': 'Traditional',
    '等別': 'Grade',
    '級別': 'Level',
    '情境': 'Context',
    '書面字頻（每百萬字）': 'WFreq',
    '口語字頻（每百萬字）': 'SFreq',
})

assert list(df.ID - 1) == list(df.index)
df = df.drop(columns=['Grade'])

df['Level'] = df.Level.str.extract('^第([1-7][*]?)級$')[0]
assert sum(df.Level.isnull()) == 0

df['Traditional'] = df.Traditional.map(fix_traditional)
df.to_csv('tbcl-chars.csv', index=False)
print('tbcl-chars.csv: %d rows' % len(df))

tbcl-chars.csv: 3100 rows


In [11]:
rows = []
for row in df.to_dict(orient='records'):
    for ch in row['Traditional'].split('/'):
        row['char'] = ch
        rows.append(dict(row))

expanded_df = pd.DataFrame(rows)[['char'] + list(df.columns)]
expanded_df.to_csv('expanded/tbcl-chars.csv', index=False)
print('expanded/tbcl-chars.csv: %d rows' % len(expanded_df))

expanded/tbcl-chars.csv: 3133 rows


In [12]:
df = pd.read_excel('downloads/tbcl-grammar.xlsx').rename(columns={
    '序號': 'ID',
    '語法點': 'Grammar',
    '等別': 'Grade',
    '級別': 'Level',
    '例句': 'Example',
})

assert list(df.ID - 1) == list(df.index)
df = df.drop(columns=['Grade'])

def fix_grammar(text):
    text = text.strip()
    text = text.replace('......', '……')
    return text

df['Grammar'] = df['Grammar'].map(fix_grammar)
df['Grammar'] = df.Grammar.str.replace('1', '¹')
df['Grammar'] = df.Grammar.str.replace('2', '²')
df['Grammar'] = df.Grammar.str.replace('3', '³')
df['Grammar'] = df.Grammar.str.replace('4', '⁴')
df['Grammar'] = df.Grammar.str.replace('5', '⁵')

df['Level'] = df.Level.str.extract('^第([1-7][*]?)級$')[0]
assert sum(df.Level.isnull()) == 0

df.to_csv('tbcl-grammar.csv', index=False)
print('tbcl-grammar.csv: %d rows' % len(df))

tbcl-grammar.csv: 496 rows


In [13]:
df = pd.read_excel('downloads/tbcl-affix.xlsx').rename(columns={
    '序號': 'ID',
    '類詞綴': 'Affix',
    '語法點': 'Grammar',
    '級別': 'Level',
    '說明': 'Explanation',
    '相關詞彙': 'Examples',
})
assert list(df.ID - 1) == list(df.index)

df['Affix'] = df.Affix.str.replace('~', '～')
df['Affix'] = df.Affix.str.replace('1', '¹')
df['Affix'] = df.Affix.str.replace('2', '²')
df['Affix'] = df.Affix.str.replace('3', '³')

df['Level'] = df.Level.str.extract('^第([1-7][*]?)級$')[0]
assert sum(df.Level.isnull()) == 0

df.to_csv('tbcl-affix.csv', index=False)
print('tbcl-affix.csv: %d rows' % len(df))

tbcl-affix.csv: 73 rows


## Readings check

In [14]:
df = pd.read_csv('tbcl.csv', dtype='str').fillna('')

In [15]:
# Check readings
if os.path.exists('../cedict/syllables.csv'):
    readings_mp = {'一': set([]), '蹟': set(['jī']), '噢': set(['yǔ'])}
    syll_df = pd.read_csv('../cedict/syllables.csv', dtype='str').fillna('')
    for row in syll_df.itertuples():
        readings_mp.setdefault(row.Traditional, set()).add(row.Pinyin.lower())
        readings_mp.setdefault(row.Simplified, set()).add(row.Pinyin.lower())
    readings_mp = {x: set([y.strip().lower() for y in readings_mp[x] if y.strip()]) for x in readings_mp}
    readings_mp['不'] = set(['bù'])

    def gen_readings(trad):
        if trad == '':
            yield ''
        elif trad[0] not in readings_mp or ord(trad[0]) < 0x3E00:
            yield from gen_readings(trad[1:])
        else:
            for x in readings_mp[trad[0]]:
                for y in gen_readings(trad[1:]):
                    yield x.lower() + ("'" if y and y[0] in 'aāáǎàeēéěèoōóǒò' else '') + y

    for row in pd.read_csv('expanded/tbcl.csv', dtype='str').fillna('').itertuples():
        trad, pinyin = row.Traditional,row.Pinyin
        readings = list(gen_readings(trad))
        if re.sub('', '', pinyin.lower()) not in readings:
            print(list(row._asdict().values())[1:9], ':\t', trad, pinyin, 'vs.', readings[:min(10, len(readings))])
            #print('%s,%s,%s' % (row.Traditional, row.Pinyin, row.Pinyin))

['1115', '開玩笑', '开玩笑', '3*', 'kāi wánxiào', 'Ph', 'to play a joke', '開一個玩笑，開別人玩笑。'] :	 開玩笑 kāi wánxiào vs. ['kāiwánxiào', 'kāiwànxiào', 'kaiwánxiào', 'kaiwànxiào']
['1137', '沒想到', '没想到', '3*', 'méi xiǎngdào', 'Ph', "didn't expect", ''] :	 沒想到 méi xiǎngdào vs. ['mòxiangdao', 'mòxiangdào', 'mòxiǎngdao', 'mòxiǎngdào', 'méixiangdao', 'méixiangdào', 'méixiǎngdao', 'méixiǎngdào']
['1160', '聖誕老人', '圣诞老人', '3*', 'Shèngdàn Lǎorén', 'N', 'Santa Claus', ''] :	 聖誕老人 Shèngdàn Lǎorén vs. ['shèngdànlǎoren', 'shèngdànlǎorén', 'shèngdànlaoren', 'shèngdànlaorén']
['1572', '阿嬤', '阿嬷', '4', 'āma', '', '', ''] :	 阿嬤 āma vs. ['ēmo', 'ēmó', 'ēmā', 'āmo', 'āmó', 'āmā', 'àmo', 'àmó', 'àmā']
['1726', '打招呼', '打招呼', '4', 'dǎ zhāohū', '', '', ''] :	 打招呼 dǎ zhāohū vs. ['dázhāohu', 'dázhāohū', 'dazhāohu', 'dazhāohū', 'dǎzhāohu', 'dǎzhāohū']
['1754', '拉肚子', '拉肚子', '4', 'lā dùzi', '', '', ''] :	 拉肚子 lā dùzi vs. ['lādùzi', 'lādùzǐ', 'lādǔzi', 'lādǔzǐ', 'lāduzi', 'lāduzǐ', 'ladùzi', 'ladùzǐ', 'ladǔzi', 'ladǔzǐ']
['2889'

## Merge with CEDICT and generate anki deck

In [16]:
UNTONE_MP = {
    'a': 'a', 'ā': 'a', 'á': 'a', 'ǎ': 'a', 'à': 'a',
    'e': 'e', 'ē': 'e', 'é': 'e', 'ě': 'e', 'è': 'e',
    'o': 'o', 'ō': 'o', 'ó': 'o', 'ǒ': 'o', 'ò': 'o',
    'i': 'i', 'ī': 'i', 'í': 'i', 'ǐ': 'i', 'ì': 'i',
    'u': 'u', 'ū': 'u', 'ú': 'u', 'ǔ': 'u', 'ù': 'u',
    'ü': 'ü', 'ǖ': 'ü', 'ǘ': 'ü', 'ǚ': 'ü', 'ǜ': 'ü'
}

# Check if pinyin from data (py1) matches cedict's (py2)
# Optionally matching untoned vowels with tones if untone==True.
def pinyin_matches(py1, py2, hz='', untone=False, yi=False, bu=False):
    py1 = py1.lower()
    py2 = py2.lower()
    i, j = 0, 0
    while i < len(py1) or j < len(py2):
        a = ''
        if i < len(py1):
            a = py1[i]
            if a in "-',/() ":
                i += 1
                continue

        b = ''
        if j < len(py2):
            b = py2[j]
            if b in "-',/() ":
                j += 1
                continue

        match = (a == b)
        match |= untone and (UNTONE_MP.get(a, a) == b or a == UNTONE_MP.get(b, b))
        if i > 0 and j > 0:
            match |= yi and py1[i-1:i+1] in ['yí', 'yì'] and py2[j-1:j+1] == 'yī' and '一' in hz
            match |= bu and py1[i-1:i+1] == 'bú' and py2[j-1:j+1] == 'bù' and '不' in hz

        if match:
            i += 1
            j += 1
        else:
            return False

    return i == len(py1) and j == len(py2)

In [17]:
df = pd.read_csv('tbcl.csv', dtype='str').fillna('')
cedict_df = pd.read_csv('../cedict/cedict.csv')
cedict_idx_mp = cedict_df.assign(idx=cedict_df.index).groupby('Traditional').idx.apply(list)

rows = []

for row in df.fillna('').to_dict(orient='records'):
    pinyin_set = set([row['Pinyin']])
    matches = cedict_idx_mp.get(row['Traditional'], [])
    if len(matches) == 0 and row['Variants']:
        variants = json.loads(row['Variants']) if row['Variants'] else [{}]
        for variant in variants:
            matches.extend(cedict_idx_mp.get(variant['Traditional'], []))
            pinyin_set.add(variant['Pinyin'])

    matches = list(sorted(set(matches)))

    flag = ''
    if len(matches) != 0:
        # Prioritize pronunciation matches, downpriorize names and variants
        # TODO: match based on taiwanese pronunciation
        if len(matches) > 1:
            matches.sort(key=lambda i: (
                -int(any(pinyin_matches(py, cedict_df.Pinyin[i], untone=False) for py in pinyin_set))
                -int(any(pinyin_matches(py, cedict_df.Pinyin[i], untone=True) for py in pinyin_set))
                +10*int(re.match('^variant', cedict_df.Definitions[i]) is not None)
                +100*int(cedict_df.Pinyin[i][0].isupper())
                +100*int(re.match('^surname', cedict_df.Definitions[i]) is not None)
            ))

        ce_simp = set([cedict_df.Simplified[i] for i in matches])
        cc_simp = opencc_tw2s.convert(row['Traditional'])
        if not row['Variants'] and ce_simp:
            if row['Simplified'] not in ce_simp:
                print('Simplified diff:', row, 'ce', ce_simp, 'cc', cc_simp)
            if len(ce_simp) > 1:
                print('Ambiguous simplified:', row, 'ce', ce_simp, 'cc', cc_simp)

        defs = []
        for i in matches:
            py1 = list(pinyin_set)[0] if len(pinyin_set) == 1 else ''
            defn = cedict_df.Definitions[i]
            #if cedict_df.Pinyin[i][0].isupper() and len(defs) == 0 and (len(py1) == 0 or not py1[0].isupper()):
            #    print('%s,%s,%s' % (row['Traditional'], py1, py1))
            defn = re.sub(r'/CL:個\|个\[ge4\](|/.*)$', r'\1', defn)  # uninformative
            if row['Variants']:
                defn = '%s [%s] %s' % (cedict_df.Traditional[i], cedict_df.Pinyin[i], defn)
            elif not pinyin_matches(py1, cedict_df.Pinyin[i], untone=False):
                defn = '[%s] %s' % (cedict_df.Pinyin[i], defn)
            defs.append(defn)

        if not row['Meaning']:
            row['Meaning'] = '<br> '.join(defs)

    #row['Flag'] = flag
    rows.append(row)

merged_df = pd.DataFrame(rows)
#merged_df = merged_df.drop(columns=['Compounds', 'Examples'])
merged_df.to_csv('tbcl-cedict.csv', index=False)

Ambiguous simplified: {'ID': '85', 'Traditional': '妳', 'Simplified': '你', 'Level': '1', 'Pinyin': 'nǐ', 'POS': 'N', 'Meaning': 'you (female)', 'Compounds': '', 'Examples': '妳好，我是妳的同學。', 'WFreq': '463', 'SFreq': '1653', 'MOE': '12580', 'Variants': ''} ce {'奶', '你'} cc 妳
Ambiguous simplified: {'ID': '853', 'Traditional': '乾', 'Simplified': '干', 'Level': '3', 'Pinyin': 'gān', 'POS': 'Vs/N', 'Meaning': 'dry / dried food', 'Compounds': '魚乾，肉乾，葡萄乾。', 'Examples': '今天早上洗的衣服，下午就乾了。 / 最近的天氣很乾，都沒有下雨。 / 我喜歡吃水果乾。', 'WFreq': '50', 'SFreq': '99', 'MOE': '16341 25228', 'Variants': ''} ce {'乾', '干'} cc 干
Simplified diff: {'ID': '1482', 'Traditional': '牠', 'Simplified': '它', 'Level': '4', 'Pinyin': 'tā', 'POS': '', 'Meaning': '', 'Compounds': '', 'Examples': '', 'WFreq': '343', 'SFreq': '659', 'MOE': '9925', 'Variants': ''} ce {'牠'} cc 牠
Simplified diff: {'ID': '3615', 'Traditional': '藉口', 'Simplified': '借口', 'Level': '5', 'Pinyin': 'jièkǒu', 'POS': '', 'Meaning': '', 'Compounds': '', 'Examples': '', 'W

*Taiwan TBCL wordlist (Traditional)*

TBCL (Taiwan Benchmarks for the Chinese Language) wordlist, 14425 words over 7 levels. Parsed from official excel sheets from [TBCL](https://coct.naer.edu.tw/TBCL/) website, including definitions/examples for about 1500 lower level words that they provide. CC-CEDICT definitions for the rest.

Pinyin normalized to not indicate tone changes for 一 and 不 for ease of joining with other data sources.

In [18]:
import genanki, shutil

df = merged_df.copy().fillna('')

!mkdir -p downloads/media
!cp -f ../downloads/fonts/MoeStandardKai.ttf downloads/media/_MoeStandardKai.ttf

cols = ['ID', 'Traditional', 'Simplified', 'Pinyin', 'Level',
        'POS', 'Meaning', 'Compounds', 'Examples', 'Variants']

model = genanki.Model(
    1698579990,
    'TBCL',
    fields=[{'name': c} for c in cols],
    templates=[{
        'name': 'TBCL',
        'qfmt': open('../dangdai/dangdai-qfmt.html').read().replace('{{ID}}', 'TBCL L{{Level}}'),
        # TODO fix template
        'afmt': '''{{FrontSide}}
<hr id=answer>
<div lang="en"><span id="ddzw-pinyin">{{Pinyin}}</span></div><br>
<div lang="en">{{#POS}}({{POS}}) {{/POS}}{{Meaning}}</div><br>
<div>{{#Compounds}}{{Compounds}}<br>{{/Compounds}}{{Examples}}</div><br>
''' + re.sub('^.*<script>', '<script>', open('../dangdai/dangdai-afmt.html').read(), flags=re.M).replace(
            'if (pinyinEl && hanziEl)',
            'if (pinyinEl && hanziEl {{#Variants}}&& false{{/Variants}})'),
    }],
    css=open('../dangdai/dangdai.css').read(),
)

deck = genanki.Deck(1698579991, name='tbcl')

for row in df.reset_index().to_dict(orient='records'):
    note = genanki.Note(
        model=model,
        fields=[row[c] for c in cols],
        guid=genanki.guid_for('tbcl', row['ID']),
        tags=['L%s' % row['Level'][0]],
    )
    deck.add_note(note)

!rm -f tbcl.apkg
genanki.Package(deck, media_files=glob.glob('downloads/media/*')).write_to_file('tbcl.apkg')
!ls -l tbcl.apkg

-rw-r--r-- 1 jovyan users 156062443 Apr 15 00:19 tbcl.apkg
